This notebook uses the python api for [Google Earth Engine](https://earthengine.google.com/), a cloud-based remote sensing platform with access to petabytes of open-source remote sensing products. Data sets are already georeferenced and ready to go. Processing occurs on Google's servers which allows users to do planetary-scale analysis from a laptop.

Google Earth Engine is free to use, but you must sign up for an account with your TAMU address. It typically takes a day or two to get apporved. 

GEE was first launched with a JavaScript API, but a python API has been developed since. I found [Qiusheng Wu's](https://github.com/giswqs/earthengine-py-notebooks) github very helpful in developing this notebook.

In this notebook, I access [Landsat 5](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C01_T1_SR) imagery and the [National Elevation Dataset (NED) DEM](https://developers.google.com/earth-engine/datasets/catalog/USGS_NED), and clip them to the [Lower Brazos Watershed](https://developers.google.com/earth-engine/datasets/catalog/USGS_WBD_2017_HUC04) - all accessed remotely from the [GEE repo](https://developers.google.com/earth-engine/datasets). And because this is all just displayed on a simple folium map, I can also add my Lower Brazos centerline from a local geoJSON file. 


In [12]:
import folium
import geojson

import ee
import geehydro

# This initializes your access to Google Earth engine resources
ee.Initialize()

The `ee` and `geehydro` packages are specific for Google Earth Engine. `ee` contains all the functions used by GEE and `geehydro` allows for a convenient integration of GEE with `folium`.

## Brazos Example

In [134]:
# Create Map
Map = folium.Map(location = (30.5, -96), tiles = 'Stamen Terrain',  zoom_start=7)

# --------------------------------------------------------------------

# Brazos River Centerline
with open('LwrBrazos_Centerline_4326.geojson') as f:
    brazos = geojson.loads(f.read())

# Strip coords out of json file and flip lat/long, wrap in tuples
tupes = [tuple(feature['geometry']['coordinates'][::-1]) for feature in brazos.features]

# Make Polyline of centerline
river = folium.PolyLine(locations=tupes)

# --------------------------------------------------------------------

# Load Lower Brazos Watershed from GEE repo
huc04 = ee.FeatureCollection('USGS/WBD/2017/HUC04')
lwr_b = huc04.filter(ee.Filter.eq('name', 'Lower Brazos'))

# Watershed visualization parameters
styleParams = {
  'fillColor': '00000000',
  'color': '0000be',
  'width': 2.0,
}

# Apply Style Parameter to FeatureCollection object
subregions = lwr_b.style(**styleParams) 

# --------------------------------------------------------------------

# Query Landsat 5 Imagery 
# Steps:
#  - Link to collection
#  - Filter scenes by area
#  - Filter by date
l5_collection = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
    .filterBounds(lwr_b) \
    .filterDate('1990-01-01', '1990-12-31')

# Flatten Image collection to sigle image
#  - take median DN for every pixel of every band
#  - clip scenes to watershed boundaries
l5_img = l5_collection.median().clip(lwr_b)

# Landsat 5 visualization parameters
l5_vis = {
    'bands':['B3', 'B2', 'B1'],
    'min': 350,
    'max': 2200
}
# --------------------------------------------------------------------

# National Elevation Dataset
dem = ee.Image('USGS/NED').select('elevation').clip(lwr_b)

# DEM visualization Parameters
dem_vis = {
    'min': 0,
    'max': 500
}
# --------------------------------------------------------------------

# Add Layers
Map.addLayer(dem, dem_vis, 'Elevation')
Map.addLayer(subregions, {}, 'Lower Brazos Watershed')
Map.addLayer(l5_img, l5_vis, 'Landsat 5 Imagery')

Map.add_child(river)

# Show Map
Map